# Lecture 10

Lecture 10 will cover decorators and error handling.

Reference
 * [2] Section 9.8-9.10
 
See also:
- https://www.datacamp.com/community/tutorials/decorators-python
- https://realpython.com/primer-on-python-decorators/
- https://www.programiz.com/python-programming/exception-handling
- https://docs.python.org/3/tutorial/errors.html
 

# Static methods

Static methods are methods that don't have access to the object itself (no `self` argument).

Why are these needed?

Usually helper functions that "belong" to the class itself (i.e. are independent of actual instances)

In [1]:
class Car:
    america = True

    @staticmethod
    def kph2mph(speed):
        return speed / 1.60934

    def __init__(self, brand, model, vel):
        self.brand = brand
        self.model = model
        self.velocity = vel

    def __str__(self):
        speed = self.velocity
        unit = "kph"
        if self.america:
            speed = self.kph2mph(speed)
            unit = "mph"
        return f"Model: {self.brand} {self.model} going at {round(speed, 2)}{unit}"

In [2]:
car1 = Car("Tesla", "Roadster", 402)
print(car1)

Model: Tesla Roadster going at 249.79mph


In [3]:
Car.america = False
print(car1)

Model: Tesla Roadster going at 402kph


In [4]:
Car.kph2mph(402)

249.7918401332223

In [5]:
# Can be called on the object as well
car1.kph2mph(402)

249.7918401332223

# Class methods

Class methods do not get `self` but they get the type of class that got used to call the method on.
This can be used to create "factory methods" in the base class that return an instance of the subclass the method is called on):

In [8]:
class Car:
    mph_kmp_conversion_factor = 1.60934
    
    @staticmethod
    def mph2kmp(speed):
        return speed * Car.mph_kmp_conversion_factor

    def __init__(self, vel):
        """
        Creates a Car
        
        :param vel: Top speed of car in kph
        """
        self.velocity = vel
        
    @classmethod
    def from_mph(cls, velocity):
        print(cls)
        new_velocity = Car.mph2kmp(velocity)
        return cls(new_velocity)

class Tesla(Car):
    pass

class Volvo(Car):
    pass

In [9]:
t = Tesla.from_mph(60)
print("----")
print(type(t))
print(t.velocity)

<class '__main__.Tesla'>
----
<class '__main__.Tesla'>
96.5604


In [10]:
v = Volvo.from_mph(55)
print("----")
print(type(v))
print(v.velocity)

<class '__main__.Volvo'>
----
<class '__main__.Volvo'>
88.5137


# Decorators

We've seen three uses of decorators already:

```python
    @staticmethod
    def kph2mph(speed):
        return speed / 1.60934
    
    @classmethod
    def from_mph(cls, velocity):
        new_velocity = cls.mph2kmp(velocity)
        return cls(new_velocity)
        
    @abc.abstractmethod
    def calories(self):
        """ Returns the number of calories in the pizza """
```

In the broadest terms, decorators modify functionality. The code using `@`:
```python
@my_decorator
def my_function():
    return compute_stuff()
```
is basically the same as doing:
```python
def my_function():
    return compute_stuff()
my_function = my_decorator(my_function)
```

We can make a silly example of what it could do:

In [ ]:
def always_print_hello(f):
    def wrapper(*x):
        print("Hello!")
        return 1+1
    return wrapper

In [ ]:
@always_print_hello
def my_function(x, y):
    return x+y+7

my_function(3,1) + my_function(3,2)

In [ ]:
@always_print_hello
def my_function3(x):
    return x+7

my_function3(3) + my_function3(3)

In [ ]:
# Achieves the same as:
def my_function2(x):
    return x+7
my_function2 = always_print_hello(my_function2)

my_function2(3) + my_function2(3)

The usefulness of decorators are perhaps not immediately obvious. But with some ingenuity they can be used to enrich the language itself. For example, enforcing an `IntEnum` to be unique:

In [ ]:
import enum
@enum.unique
class GameState(enum.IntEnum):
    not_started = 0
    started = 1
    ended = 2
    paused = 2 # Opps

You will not be required to write any decorators on the exam, though you should know about `@staticmethod` and `@abc.abstractmethod` and how these are used.
Abstract and static methods are re-occuring themes in object oriented languages, though they might be expressed differently (i.e. using different syntax) in different languages.

Other uses of decorators are more unique "language quirks", and excessive use just makes the code hard to understand, so don't go crazy with them!

Extension: Decorators taking arguments:

In [ ]:
def always_print_x(x):
    def _always_print_x(f):
        def wrapper(*z):
            print(f"This is x: {x}")
            return f(*z)
        return wrapper
    return _always_print_x

@always_print_x("foo")
def sumit(a, b):
    return a + b

sumit(1,2)

### Excercises:
    
- Create a decorator (`@assert_postitive`) that makes a function raise an exception if the returned value is not an integer or if it is less than zero.

# Error handling

When dealing with user input (or even programmer input, in the example above), errors are inevitable.
We can't always know if a given piece of code will always work (i.e. cause an error on certain user inputs)

The way to deal with this is to *try* to execute segments, and deal with the errors if they occur:

In [ ]:
1 / y

In [ ]:
for x in range(2,-1,-1):
    print(x)
    print(1.0 / x)

In [ ]:
for x in range(2,-1,-1):
    print(x)
    try:
        print(1.0 / x)
    except:
        print("inf")

Though, we probably want to check *what* error occured, and deal with each specifically

In [ ]:
for x in range(2,-1,-1):
    try:
        print(1.0 / y) # Opps, wrong variable, not defined
    except:
        print("inf") # Not correct for this error

In [ ]:
for x in range(2,-1,-1):
    try:
        print(1.0 / x) # Opps, wrong variable, not defined
    except ZeroDivisionError:
        print("inf")

In [ ]:
stuff = [1,2,3,4]
for x in range(8,-1,-1):
    try:
        print(stuff[x]) # Opps, wrong variable again
    except ZeroDivisionError:
        print("inf")
    except NameError:
        print("Wrong symbol!")
    except IndexError:
        print("Problematic index")

In [ ]:
x = 6

[1,2,3][x]

This is why broadly catching all errors are very bad form. Don't do it! Also, catching things like `NameError` is rarely a good idea, better to let Python error out because there is a bug in the program.

### Using "as" to obtain info from errors

In [ ]:
import sys

try:
    f = open('filename.txt')
    s = f.readline()
    i = int(s.strip())
except FileNotFoundError as e:
    print(f"Failed to open file {e.filename}, please try again")

In [ ]:
help(FileNotFoundError)

### Finally and else

In [ ]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
        return 0 # finally will run even if function exits here
    else:
        # Will run if we don't have an exception first:
        print("result is", result)
    finally:
        # This will always be executed last
        print("executing finally clause")

In [ ]:
divide(0,0)

In [ ]:
divide(2,3)

## Error propagation

When an error is thrown, Python will "unwind the stack" (look upwards through the callers) until it find an `except` that handles the exception. Execution will continue at that point.

The fact that execution can move like this can make code with a lot of `try except` very hard to understand. Use it sparingly!

In [ ]:
def f1():
    try:
        f2()
    except:
        print("errored while calling f2")
    
    
def f2():
    try:
        f3()
    except:
        print("errored while calling f3")
        raise Exception
        

def f3():
    f4()


def f4():
    raise Exception
    

f1()

#                 (boom #2)
# f1 --------------> f2 -----> f3 -------> f4 (boom) # normal exectuion
#  ^--[exception]---v ^-----[exception]-----v     # exception unwinding

# Making your own errors

An exception is just a class (that should inherit from Exception or one of its subclasses):

In [ ]:
class MyCustomError(Exception):
    # Inheriting from the base class for all exceptions
    pass

class NegativeValueError(ValueError):
    # Subclassing an exception to add a useful specific case
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return "Value is " + repr(self.value)

Catching the exception you can get any stored values like any object:

In [ ]:
def my_function(x):
    if x < 0:
        raise NegativeValueError(x)
    return x * 3

In [ ]:
# x = 3
x = -2
try:
    print(my_function(x))
except NegativeValueError as e:
    print('Problem with x={}, try again.'.format(e.value))

Uncaught exceptions prints the string representations of the class:

In [ ]:
my_function(-2)

You should know when to use the most common built in exceptions, e.g.
- ValueError - when the *value* isn't right
- TypeError - when the *type* isn't right
- IndexError - when index/item isn't in the container (i.e. out of range index on a vector) 

or which ones to inherit in your custom exception on (if appropriate)

## Possible custom exceptions

**Question**
* In your Poker-game library, what would be some suitable exceptions to implement?

---------------------

---------------------

---------------------


---------------------


---------------------


---------------------


---------------------


---------------------


---------------------


---------------------


---------------------

* `MissingCardError`
* `OutOfMoneyError`
* `EmptyDeckError`